### Let's get the styling out of the way.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### We are going to model point earthquake and update the survival probability into the graph database.
Let's first create neo4j instance and inject point earthquake scenario.

In [ ]:
from erad.db import neo4j_
neo4j_instance = neo4j_.Neo4J()

In [ ]:
import datetime
import shapely

from erad.scenarios import earthquake_scenario
from erad.db import inject_scenario
from erad.db import scenario_model


scenario_input = scenario_model.PointEarthquake(
        longitude=-121.72125955960196, 
        latitude=37.92770173811863,
        probability_model={},
        timestamp=datetime.datetime(2022, 1, 1, 0, 0, 0),
        magnitude=8.0,
        depth=30.0,
    )
inject_scenario.inject_point_earthquake(
    scenario_input, neo4j_instance.driver
)

### Let's visualize the survival probability and see which AC line segment survive and which did not.

In [ ]:
cypher_query = """
                    MATCH (b1:Bus)-[r:CONNECTS_TO]-(b2:Bus)
                    WHERE r.ampacity IS NOT NULL
                    RETURN r.longitude AS longitude, r.latitude AS latitude , r.name, r.survival_probability AS survival_probability, r.survive AS survive
                """
with neo4j_instance.driver.session() as session:
    result = session.read_transaction(
        lambda tx: tx.run(cypher_query).data()
    )

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame(result)
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='survival_probability',
                        center=dict(lat=37.892652, lon=-121.723465), zoom=16,
                        mapbox_style="carto-darkmatter")
fig.show()
df['survive_str'] = [str(el) for el in list(df['survive'])]
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="r.name",
                        color="survive_str", zoom=16, center=dict(lat=37.892652, lon=-121.723465), mapbox_style="carto-darkmatter")
fig.show()

### Let's see which customers are getting power and which are not

In [ ]:
from erad.metrics import metric
metric.is_customer_getting_power(
        neo4j_instance.driver, "./is_customer_connected.csv"
    )

In [ ]:
metric_df = pd.read_csv('./is_customer_connected.csv')
cypher_query = """
                    MATCH (lo:Load)
                    RETURN lo.longitude AS longitude, lo.latitude AS latitude , lo.name
                """
with neo4j_instance.driver.session() as session:
    result = session.read_transaction(
        lambda tx: tx.run(cypher_query).data()
    )
load_df = pd.DataFrame(result)
df = metric_df.merge(load_df, left_on="load_name", right_on="lo.name")
df['metric_str'] = [str(el) for el in list(df['metric'])]
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="lo.name",
                        color="metric_str", zoom=16, center=dict(lat=37.892652, lon=-121.723465), mapbox_style="carto-darkmatter")
fig.show()

### Energy resilience metric for customers

In [ ]:
metric.energy_resilience_by_customer(
        neo4j_instance.driver, "./energy_resilience.csv"
    )

In [ ]:
metric_df = pd.read_csv('./energy_resilience.csv')
metric_df = metric_df[metric_df['critical_service']=='Grocery']
df = metric_df.merge(load_df, left_on="load_name", right_on="lo.name")
fig = px.scatter_mapbox(df, lat='latitude', lon='longitude', color='metric',
                        center=dict(lat=37.892652, lon=-121.723465), zoom=16,
                        mapbox_style="carto-darkmatter")
fig.show()